In [1]:
import os
import torch
from torch import nn
import numpy as np
import random
from dataset import ModelnetDataset
from pathlib import Path
from torch.utils.data import DataLoader
from tqdm import trange, tqdm


from util import setup_seed, train, test, train_HHN, test_HHN, train_one4one, count_parameters, save_checkpoint
from model import One4All, HyperNetwork
from render import PointcloudRender

from torch.utils.tensorboard import SummaryWriter

In [ ]:
setup_seed(666)
device_for_dataset = "cuda:3" if torch.cuda.is_available() else "cpu"


In [ ]:
path = Path("./data/ModelNet10")
train_ds = ModelnetDataset(path, folder="train", device=device_for_dataset) # Just put dataset in mem or GPU
test_ds = ModelnetDataset(path, folder="test", device=device_for_dataset) # Just put dataset in mem or GPU
classes = {i: cat for cat, i in train_ds.classes.items()}

print('Train dataset size: ', len(train_ds))
print('Test dataset size: ', len(test_ds))
print('Number of classes: ', len(train_ds.classes))
print('Class: ', classes[train_ds[11]['category']])

In [ ]:
from collections import Counter
print(dict(Counter(sample['category'] for sample in train_ds.files)))

print(dict(Counter(sample['category'] for sample in test_ds.files)))



In [ ]:
batch_size = 64 #  
epochs = 100 #
best_acc1 = 0
start_epoch = 0


In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=True)


In [ ]:
gpu_computation = "cuda:0" if torch.cuda.is_available() else "cpu"

pr = PointcloudRender(              
                 img_size=128,
                 radius=0.0045,
                 points_per_pixel=40)




In [ ]:
torch.cuda.empty_cache()


# angles = torch.randint(-180, 180, (3,))/180*torch.pi

test_acc_list = []
degrees = []
# Average 216 one-for-one models
# for x in range(-180, 181, 72):
# 	for y in range(-180, 181, 72):
# 		for z in range(-180, 181, 72):
# 			angles = torch.tensor([x,y,z])/180*torch.pi

# Average 30 one-for-one models
for i in tqdm(range(30), desc="Training one4one"):
    angles = torch.randint(-180, 180, (3,))
    x,y,z = angles
    print(f'Angles: {angles}')
    angles = angles/180*torch.pi
    
    writer = SummaryWriter(f'tf-logs/one4one/01-11/{x}_{y}_{z}')

    model = One4All() 
    model = model.to(gpu_computation)
    # print(f'Num of model parameter: {count_parameters(model)}')

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay = 1e-6)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=5e-6)

    for t in range(start_epoch, epochs):
        # print(60*"="+f"\n Epoch: {t + 1} \n"+60*"=")

        train_acc, train_loss= train_one4one(train_loader, model, loss_fn, optimizer, t, pr, device=gpu_computation, angles=angles, verbose=False)
        writer.add_scalar("Loss/train", train_loss, t)
        writer.add_scalar("Accuracy/train", train_acc, t)
        test_acc, test_loss = test(test_loader, model, loss_fn, pr, device=gpu_computation, angles=angles, verbose=False)

        if (t+1)%60 == 0:
            test_acc, test_loss = test(test_loader, model, loss_fn, pr, device=gpu_computation, angles=angles, verbose=False)

        is_best = test_acc > best_acc1
        best_acc1 = max(test_acc, best_acc1)
        
        if (t+1)%60 == 0:


            save_checkpoint({
            'epoch': t,
            'state_dict': model.state_dict(),
            'best_acc1': best_acc1,
            'optimizer': optimizer.state_dict(),
            }, is_best,
            filename= f'checkpoint/one4one/01-11/E{t}-{x}-{y}-{z}-one4onecheckpoint.pth.tar',
            best_filename = f'checkpoint/one4one/01-11/best-{x}-{y}-{z}-one4oneheckpoint.pth.tar'
            )
            test_acc_list.append(test_acc)
            degrees.append(angles)
        
        writer.flush()
        lr_scheduler.step()

    writer.close()
test_acc_list = np.array(test_acc_list)
np.save("checkpoint/one4one/01-11/average_acc.npy", test_acc_list)
np.save("checkpoint/one4one/01-11/angles.npy", angles)

print(f"{test_acc_list.mean()}")


In [ ]:
print(np.load("checkpoint/one4one/01-11/average_acc.npy").mean())

In [3]:
print(np.load("accuracy_result_npy/LeNet5/one4one_acc.npy").mean())

0.8062408228715261
